In [ ]:
from regression.lightgbm.model import LightGBM
from regression.lightgbm.loss import rmspe,feval_rmspe,rmse

In [2]:
import pandas as pd

In [3]:
train=pd.read_csv("regression/lightgbm/dataset/train.csv")

In [4]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [5]:
target_feature="Survived"

In [8]:
from typing import Tuple, Union, Dict, List, Optional
def split_cabin_str(s: str) -> Tuple[str, float]:
    """
    Function to try and extract cabin letter and number from the cabin column.
    Runs a regular expression that finds letters and numbers in the
    string. These are held in match.group, if they exist.
    """

    match = re.match(r"([a-z]+)([0-9]+)", s, re.I)

    letter = ''
    number = -1.0
    if match is not None:
        letter = match.group(1)
        number = match.group(2)

    return letter, number


def process_cabin(s: Union[str, float]) -> Tuple[str, float, int]:
    """From a cabin string, try and extract letter, number, and number of cabins."""
    # Check contents
    if not isinstance(s, str):
        # If field is empty, return nothing
        letter = ''
        number = ''
        n_rooms = -1.0
    else:
        # If field isn't empty, split sting on space. Some strings contain
        # multiple cabins.
        s = s.split(' ')
        # Count the cabins based on number of splits
        n_rooms = len(s)
        # Just take first cabin for letter/number extraction
        s = s[0]

        letter, number = split_cabin_str(s)

    return letter, number, n_rooms


def split_name_str(s: str, title_map: Dict[str, str]) -> Tuple[str, str]:
    """
    Extract title from name, replace with value in title dictionary and both title 
    and surname.
    """

    # Remove '.' from name string
    s = s.replace('.', '')
    # Split on spaces
    s = s.split(' ')
    # get surname
    surname = s[0]

    # Get title - loop over title_map, if s matches a key, take the
    # corresponding value as the title
    title = [t for k, t in title_map.items() if str(k) in s]

    # If no matching keys in title dict, use 'Other'.
    if len(title) == 0:
        title = 'Other'
    else:
        # Title is a list, so extract contents
        title = title[0]

    # Return surname (stripping remaining ',') and title as string
    return surname.strip(','), title

In [13]:
import gc
import re
class LightGBM_model1(LightGBM):
    def _featuresEngineering(self):
        cabins = self.data['Cabin'].apply(process_cabin)
        # Output tuple has 3 values for each row, convert this to pandas df
        cabins = cabins.apply(pd.Series)
        # And name the columns
        cabins.columns = ['cabin_letter', 'cabin_number', 'cabins_number_of']
        # Then concatenate these columns to the dataset
        self.data = pd.concat([self.data, cabins], axis=1)
        
        
        # Then concatenate these columns to the dataset
        self.data = pd.concat([self.data, cabins], axis=1)

        # Process family
        # Add some family features directly to new columns in the dataset
        # Size
        self.data.loc[:, 'fSize'] = self.data['SibSp'] + self.data['Parch'] + 1
        # Ratio
        self.data.loc[:, 'fRatio'] = ((self.data['Parch'] + 1) 
                                        / (self.data['SibSp'] + 1))
        # Adult?
        self.data.loc[:, 'Adult'] = self.data['Age'] > 18

        # Process titles
        # Extract titles from Name column, standardise
        title_map_ = {"Capt": "Officer", "Col": "Officer", "Major": "Officer", 
                      "Jonkheer": "Sir", "Don": "Sir", "Sir": "Sir", "Dr": "Dr", 
                      "Rev": "Rev", "theCountess": "Lady", "Dona": "Lady", 
                      "Mme": "Mrs", "Mlle": "Miss", "Ms": "Mrs", "Mr": "Mr", 
                      "Mrs": "Mrs", "Miss": "Miss", "Master": "Master",
                      "Lady": "Lady"}

        # Apply functions to df and concatenate new columns as before
        cabins = self.data['Name'].apply(split_name_str, args=[title_map_])
        cabins = cabins.apply(pd.Series)
        cabins.columns = ['Surname', 'Title']

        self.data = pd.concat([self.data, cabins], axis=1)

        
        
        
        
        cat_cols = ['Sex', 'Embarked', 'cabin_letter', 'cabin_number', 'Surname', 
                'Title']

        # Recode
        for c in cat_cols:
            # Convert column to pd.Categorical
            self.data.loc[:, c] = pd.Categorical(self.data[c])
            # Extract the cat.codes and replace the column with these
            self.data.loc[:, c] = self.data[c].cat.codes
            # Convert the cat codes to categorical...
            self.data.loc[:, c] = pd.Categorical(self.data[c])
        del cabins
        
        # Age
        # Replace missing age values with median.
        # See other kernels for more sophisticated ways of doing this!
        self.data.loc[self.data.Age.isnull(), 'Age'] = \
            np.median(self.data['Age'].loc[self.data.Age.notnull()])
        
        gc.collect()
        
lgb=LightGBM_model1(train,target_feature,rmse)

In [14]:
lgb._train(kfold_n=5)

AttributeError: 'DataFrame' object has no attribute 'cat'